# Final Combined Version of the code with UI

In [2]:
# Create tkinter UI to select folder
#Import tkinter library
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
import OpenDir
import os
import glob
import ntpath
import sys
import time 
import pandas as pd
#csv to png imports
from datetime import datetime
import csv
import matplotlib.pyplot as plt


# create the root window
root = tk.Tk()
root.title('Tkinter File Dialog')
root.resizable(False, False)
root.geometry('600x300')


def get_TRC_list():
    filetypes = (
        ('Log Files', '*.trc'),
        ('All files', '*.*')
    )

    filenames = fd.askopenfilenames(
        title='Open Trc Files',
        initialdir='/Test_Data/',        
        filetypes=filetypes)

    showinfo(
        title='Selected Files',
        #message print the number of files selected
        message=f'{len(filenames)} files selected'
    )
    #return the list of files selected
    return filenames
        
#call function to select files
def get_CSV_list():
    filetypes = (
        ('CSV Files', '*.csv'),
        ('All files', '*.*')
    )

    filenames = fd.askopenfilenames(
        title='Open CSV Files',
        initialdir='/Test_Data/',        
        filetypes=filetypes)

    showinfo(
        title='Selected Files',
        #message print the number of files selected
        message=f'{len(filenames)} files selected'
    )
    #return the list of files selected
    return filenames



#create a global variable to store the folder directory derived from the gui_fname function
dirName = ""
headers = ['Tonneau_Hall_Counts', 'ForwardCurrentx100', 'ReverseCurrentx100']

# #display gui_fname() button



def to_little(val):
    little_hex = bytearray.fromhex(val)
    little_hex.reverse()
    #print("Byte array format:", little_hex)

    str_little = ''.join(format(x, '02x') for x in little_hex)

    return str_little


def main():
    # #assign variable file_list to function get_file_list()
    dirName = ""
    trc_list = get_TRC_list()
    N=len(trc_list)

    #Print number of files in the folder
    print(f'Found {N} data files\n')

######Loop through all files in the folder######
    for i, f in enumerate(trc_list):
        print('---------------------------------')
        print(f'{i+1}/{N} Processing "{f}"!')
        trc_list = trc_list[:] #process all files on the list
        try:
            if i < N:   

                df = pd.read_csv(trc_list[i], comment=';', names=["All"])
            #pd.set_option('display.max_rows', None)
                df = df[df.All.str.contains("076C")] #filter out invalid data
            #create columns TimeStamp from column 12:20
                df2 = pd.DataFrame()
                #df2['TimeStamp'] = df['All'].str[11:20]
                df2['Tonneau_ForwardCurrent'] = df['All'].str[41:47]
                df2['Tonneau_ReverseCurrent'] = df['All'].str[47:53]
                df2['Tonneau_Hall_Counts'] = df['All'].str[53:59]


            #data frame to convert hexadecimal to decimal 
                df3 = pd.DataFrame()
                df2['ForwardCurrentDecimal'] = df2['Tonneau_ForwardCurrent'].apply(lambda x: int(to_little(x), 16))
                df2['ReverseCurrentDecimal'] = df2['Tonneau_ReverseCurrent'].apply(lambda x: int(to_little(x), 16))

                df3['Tonneau_Hall_Counts'] = df2['Tonneau_Hall_Counts'].apply(lambda x: int(to_little(x), 16))
            #create data frame that reverses the hex byte order

            #create dataframe, multiply by 10 to get the value in mA
                df3['ForwardCurrentx100'] = df2['ForwardCurrentDecimal'] * 10
                df3['ReverseCurrentx100'] = df2['ReverseCurrentDecimal'] * 10

            #convert Tonneau ForwardCurrent from hexadecimal string to integer
                print(df3)

        #save csv file to subdirectory called "csv" folder
                if not os.path.exists(dirName + "CSV"):
                    os.makedirs(dirName + "CSV")

                df3.to_csv(dirName + "csv/" + ntpath.basename(trc_list[i] + '.csv'), index=False)
        except:
            print("Invalid Hex Format Found")

def OpenFiles():
    #assign variable file_list to function get_file_list()
    trc_list = get_TRC_list()


process_button = ttk.Button(
    
    root,
    text='Process trc Files',
    command=main
)


def main_PNG():
    start_t = datetime.now()
    dirName = ""
    csv_list = get_CSV_list()

    N=len(csv_list)
    #sleep for 3 seconds
    print(f'Found {N} .csv files\n')

    for i, f in enumerate(csv_list):
        print('---------------------------------')
        print(f'{i+1}/{N} Processing "{f}"!')
        csv_list = csv_list[:] #process all files on the list

        # run the for loop until the end of the file_list else continue
        
        fh = open(csv_list[i])
        csv_reader = csv.reader(fh)
        csv_header = next(csv_reader)
        csv_header

        # print(df_sig)

        lst_dt_csv = next(csv_reader)
        lst_dt_csv
        fh.close()
            #create new df_sig for each file

            #create loop to save each plot from the corresponding csv file to different png files
        if i < N:
            df_sig = pd.read_csv(csv_list[i], header=None, skiprows=1, names=headers)
            #make plot lines thinner 
            plt.rcParams['lines.linewidth'] = 0.5
            df_sig.ForwardCurrentx100.plot()
            df_sig.ReverseCurrentx100.plot()
            df_sig.Tonneau_Hall_Counts.plot()
            #plot legend above right
            plt.legend(bbox_to_anchor=(1.0, 1.0))
            plt.xlabel('Time Axis')
            plt.ylabel('Value Axis')
            #save the plot and labels to a file as png in folder called "PNG"
            if not os.path.exists(dirName + "PNG"):
                os.makedirs(dirName + "PNG")
            #save the plot as png in the folder PNG
            plt.savefig(dirName + "PNG/" + ntpath.basename(csv_list[i]) + '.png', bbox_inches='tight', dpi=300)
            plt.close()
            print("Plot saved to " + dirName + "PNG/" + ntpath.basename(csv_list[i]) + '.png')
            # plt.show()


    
CSV_button = ttk.Button(
    root,
    text='Process CSV to PNG',
    command=main_PNG
)

def folder():
    Folder = fd.askopenfilenames(
    title='View Folder',
    initialdir='/Test_Data/'
    )      

#View folder button
folder_button = ttk.Button(
root,
text='View Folder',
command=folder
)




process_button.pack(expand=True)


CSV_button.pack(expand=True)
# filenames=get_CSV_list()

# folder_button.pack(expand=True)
folder_button.pack(expand=True)


root.mainloop()

#

Found 8 data files

---------------------------------
1/8 Processing "C:/Users/aalkhafaji/Documents/ETC_Project_Python/TRC2CSV/Test_Data/20220210093758_PCAN_USBBUS1_1.trc"!
      Tonneau_Hall_Counts  ForwardCurrentx100  ReverseCurrentx100
5                    3986                  20                   0
13                   3986                  20                   0
16                   3986                  20                   0
23                   3986                  20                   0
25                   3986                  20                   0
...                   ...                 ...                 ...
9319                 3986                  20                   0
9322                 3986                  20                   0
9325                 3986                  20                   0
9328                 3986                  20                   0
9331                 3986                  20                   0

[2590 rows x 3 columns]
----------

KeyboardInterrupt: 

: 

In [2]:
from tkinter import *
from tkinter import filedialog 

